## K-means Clustering
**(With extracts from the Nicolas Vandepu's book "Data Science for Supply Chain Forecasting")**

K-means clustering is a widely utilized unsupervised learning technique in data science for classifying unlabeled data into groups based on their features, not pre-defined categories. The method involves partitioning data points into 'k' distinct non-overlapping subgroups or clusters. The number 'k' represents the number of clusters to be created, which is a parameter defined by the user. The main goal of K-means is to minimize the sum of the squared distances between the data points and their respective cluster centroid, which is the center of mass for each cluster. This algorithm is typically the first choice for practitioners to get an understanding of the dataset's structure due to its simplicity and effectiveness in revealing patterns and regularities within the data.

In [1]:
import pandas as pd

# Define the import_data function
def import_data():
    data = pd.read_csv(file_path)
    data['Period'] = data['Year'].astype(str) + '-' + data['Month'].astype(str).str.zfill(2)
    df = pd.pivot_table(data=data, values=['Quantity'], index='Make', columns='Period', aggfunc='sum', fill_value=0)
    return df

# URL of the CSV file
file_path = "https://supchains.com/wp-content/uploads/2021/07/norway_new_car_sales_by_make1.csv"

# Create the DataFrame using the import_data function
df = import_data()

# Now 'df' contains the data from the provided URL in the desired format.

# Print the DataFrame
print(df.head())

             Quantity                                                          \
Period        2007-01 2007-02 2007-03 2007-04 2007-05 2007-06 2007-07 2007-08   
Make                                                                            
Alfa Romeo         16       9      21      20      17      21      14      12   
Aston Martin        0       0       1       0       4       3       3       0   
Audi              599     498     682     556     630     498     562     590   
BMW               352     335     365     360     431     477     403     348   
Bentley             0       0       0       0       0       1       0       0   

                              ...                                          \
Period       2007-09 2007-10  ... 2016-04 2016-05 2016-06 2016-07 2016-08   
Make                          ...                                           
Alfa Romeo        15      10  ...       3       1       2       1       6   
Aston Martin       0       0  ...       0  

## Looking for Meaningful Centers
 
We just learned a compelling and automated way to label an entire dataset, thanks to machine learning. But how can we apply this to our historical demand? 
The question you should ask yourself is: what are the features I want to categorize my products on? Depending on each dataset, you might prefer different approaches. Here are some ideas: 
               
- **Volume:**
We can obviously start categorizing products based on their sales volume. Most likely, this categorization won’t provide so much added value, as you could just do a good old Pareto Classification to get a similar result. 
- **Additive Seasonality:**
If we cluster the products based on their additive seasonality factors, we will cluster them based on their average volume and their seasonality. You might then end up with groups containing just a few products. 
- **Multiplicative Seasonality:** 
If we take the multiplicative seasonal factors, our products will then be categorized only based on their seasonal behavior—irrespective of their absolute size. This sounds much better.

### Scaling
The K-means algorithm is very sensitive to extreme values (that, by definition, are far away from any other values) so that we will have to normalize all the seasonal factors. It means that each product’s seasonal factors will be reduced to have a mean of 0 and a range of 1.

K-means is very sensitive to scaling; always remember to scale your dataset before applying this technique. As we did here, it is always a best practice to visualize your results to check that they are meaningful. Another good check is to count the number of products in each cluster: if you obtain some clusters with only a few items, that might be a clue that the clusters are not meaningful.

####  Compute the (multiplicative) seasonal factorsLet’s 
In order to do so, we will create a function seasonal_factors() that will return the seasonal factors based on a historical demand dataset.

In [2]:
def seasonal_factors(df, slen):
    # Create a new DataFrame 's' with the same index as 'df' to store the seasonal factors
    s = pd.DataFrame(index=df.index)
    
    # Loop over the range of seasonal length 'slen' to calculate mean values
    # for each season across all columns
    for i in range(slen):
        # Calculate the mean of every 'slen' elements in each column and assign it
        # to a new column in DataFrame 's'. This mean is a seasonal factor.
        # 'i::slen' is a slicing technique that starts at index 'i' and skips 'slen' elements at a time.
        s[i+1] = df.iloc[:, i::slen].mean(axis=1)
    
    # Normalize each column in 's' by dividing by the mean of the column to get the relative seasonal factors
    # Then, replace any missing values resulted from division by zero with 0.
    s = s.divide(s.mean(axis=1), axis=0).fillna(0)
    
    # Return the DataFrame containing seasonal factors for each season
    return s

#### Create a scaler() function 
It will return the seasonality factors scaled with a range of 1 and a mean of 0.

In [3]:
def scaler(s):
    # Calculate the mean of each row
    mean = s.mean(axis=1)
    
    # Find the maximum value in each row
    maxi = s.max(axis=1)
    
    # Find the minimum value in each row
    mini = s.min(axis=1)
    
    # Subtract the mean from each element in the DataFrame (broadcasting along rows)
    s = s.subtract(mean, axis=0)
    
    # Perform element-wise division of the DataFrame by the range (max - min)
    # Broadcasting along rows. If the range is zero, 'fillna(0)' handles division by zero by replacing NaNs with 0.
    s = s.divide(maxi-mini, axis=0).fillna(0)
    
    # Return the scaled DataFrame
    return s

We can now use both our functions to populate scaled seasonal factors:

In [4]:
df = import_data
s = seasonal_factors(df,slen=12)
s= scaler(s)
print(s.head())

AttributeError: 'function' object has no attribute 'index'

### Training and Test Sets Creation
Now that we have our dataset with the proper formatting, we can create a function datasets() to populate a training and a test set.

In [ ]:
import numpy as np

# Define the datasets function with x_len as an argument
def datasets(df, x_len=12, y_len=1, test_loops=12):
    D = df.values
    rows, periods = D.shape
    
    # Training set creation
    loops = periods + 1 - x_len - y_len
    train = []
    for col in range(loops):
        train.append(D[:, col:col + x_len + y_len])
    train = np.vstack(train)
    X_train, Y_train = np.split(train, [-y_len], axis=1)
    
    # Test set creation
    if test_loops > 0:
        X_train, X_test = np.split(X_train, [-rows * test_loops], axis=0)
        Y_train, Y_test = np.split(Y_train, [-rows * test_loops], axis=0)
    else:
        X_test = D[:, -x_len:]
        Y_test = np.full((X_test.shape[0], y_len), np.nan)
    
    # Formatting required for scikit-learn
    if y_len == 1:
        Y_train = Y_train.ravel()
        Y_test = Y_test.ravel()
        
    return X_train, Y_train, X_test, Y_test

### Call our new function datasets(df) as well as import_data()
We can now easily call our new function datasets(df) as well as import_data(). We obtain the datasets we need to feed our machine learning algorithm (X_train and Y_train) and the datasets we need to test it (X_test and Y_test)

In [ ]:
# Import data
df = import_data()

# Create training and test sets using the datasets function
X_train, Y_train, X_test, Y_test = datasets(df, x_len=12, y_len=1, test_loops=12)

### Analyzing Feature Importance with XGBoost in Machine Learning
Machine learning algorithms often operate on large datasets with numerous features, making it essential to understand the importance of each feature in the model's predictions. XGBoost, an efficient and scalable machine learning algorithm, provides a built-in method to assess feature importance. In this code snippet, we utilize XGBoost to create a regression model and analyze the importance of features using its plotting capabilities. Let's delve into the organized version of the code to gain insights into how feature importance is visualized and interpreted in the context of machine learning.

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

# Initializing XGBoost regression model with specified parameters
XGB = XGBRegressor(n_jobs=-1, max_depth=10, n_estimators=100, learning_rate=0.2)

# Fitting the XGBoost model with training data
XGB.fit(X_train, Y_train)

# Assigning feature names to the booster object for visualization
XGB.get_booster().feature_names = [f'M{x-12}' for x in range(12)]

# Plotting feature importance using XGBoost's plot_importance function
xgb.plot_importance(XGB, importance_type='total_gain', show_values=False)

### Evaluation and Early Stopping
When an XGBoost model is trained on a dataset, you can measure—after each iteration—its accuracy against an evaluation set.

**Evaluation Set**
An evaluation set is a set of data that is left aside from the training set to be used as a monitoring dataset during the training. A validation set (random subset of the training set) or a holdout set (last period of the training set) can be used as an evaluation set.

If you want to optimize your model’s parameters, the best practice is of course to run a cross-validation random search. But rather than trying different models with a different number of trees, you could grow your model indefinitely, and stop it when there is no more improvement on the evaluation set (for some consecutive iterations).

In practice, once the model does not see an accuracy improvement on the evaluation test for a determined number of extra trees, XGBoost will revert to the last iteration that brought extra accuracy to the evaluation set. With this technique, you get rid of the burden of number-of-trees optimization, and you are sure to grow your model up to the right level. 
               This early stopping technique will help us avoid overfitting our model to the training set, and at the same time, reduce training time. One stone, two birds. The early stopping technique is a very useful capability of XGBoost.

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Splitting the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.15)

# Setting up the XGBoost regressor with specified hyperparameters
# Including the `eval_metric` and `early_stopping_rounds` during initialization
XGB = XGBRegressor(
    n_jobs=-1, 
    max_depth=10, 
    n_estimators=1000, 
    learning_rate=0.01, 
    early_stopping_rounds=100,
    eval_metric='mae'
)

# Fitting the model on the training data
# Note: No need to specify `early_stopping_rounds` and `eval_metric` here
XGB.fit(
    x_train, 
    y_train, 
    verbose=False, 
    eval_set=[(x_val, y_val)]
)

# Printing the best iteration number
print(f'Best iteration: {XGB.best_iteration}')

# Printing the best score
print(f'Best score: {XGB.best_score}')

#### Using holdout dataset as the evaluation dataset
Instead of a validation set, you can also use a holdout dataset as the evaluation dataset. This can perform better on some datasets.

In [ ]:
def datasets_holdout(df, x_len=12, y_len=1, test_loops=12, holdout_loops=0):
    D = df.values
    rows, periods = D.shape
    
    # Training set creation
    train_loops = periods + 1 - x_len - y_len - test_loops
    train = []
    for col in range(train_loops):
        train.append(D[:, col:col + x_len + y_len])
    train = np.vstack(train)
    X_train, Y_train = np.split(train, [-y_len], axis=1)
    
    # Holdout set creation
    if holdout_loops > 0:
        X_train, X_holdout = np.split(X_train, [-rows * holdout_loops], axis=0)
        Y_train, Y_holdout = np.split(Y_train, [-rows * holdout_loops], axis=0)
    else:
        X_holdout, Y_holdout = np.array([]), np.array([])
    
    # Test set creation
    if test_loops > 0:
        X_train, X_test = np.split(X_train, [-rows * test_loops], axis=0)
        Y_train, Y_test = np.split(Y_train, [-rows * test_loops], axis=0)
    else:
        # No test set: X_test is used to generate the future forecast
        X_test = D[:, -x_len:]
        Y_test = np.full((X_test.shape[0], y_len), np.nan)  # Dummy value
    
    # Formatting required for scikit-learn
    if y_len == 1:
        Y_train = Y_train.ravel()
        Y_test = Y_test.ravel()
        Y_holdout = Y_holdout.ravel()
    
    return X_train, Y_train, X_holdout, Y_holdout, X_test, Y_test

In [ ]:
# Generating training, holdout, and test sets using a custom function
X_train, Y_train, X_holdout, Y_holdout, X_test, Y_test = datasets_holdout(df, x_len=12, y_len=1, test_loops=12, holdout_loops=12)

# Initializing and training XGBoost regression model
XGB = XGBRegressor(n_jobs=-1, max_depth=10, n_estimators=2000, learning_rate=0.01)
XGB = XGB.fit(X_train, Y_train, verbose=False, eval_metric='mae')

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Predicting values for holdout and test sets
predictions_holdout = XGB.predict(X_holdout)
predictions_test = XGB.predict(X_test)

# Calculating MAE for holdout and test sets
mae_holdout = mean_absolute_error(Y_holdout, predictions_holdout)
mae_test = mean_absolute_error(Y_test, predictions_test)

# Creating a DataFrame with the results
results_data = {
    'Dataset': ['Holdout', 'Test'],
    'Mean Absolute Error (MAE)': [mae_holdout, mae_test]
}

results_df = pd.DataFrame(results_data)

# Displaying the table
print(results_df)

**Important note:** It's important to compare the MAE to some baseline or to the range of the target variable to understand the magnitude of these errors fully. Additionally, if these datasets are different in size, distribution, or feature space, the MAE values may not be directly comparable.

### Hyperparameter Tuning Grid for Gradient Boosting Algorithm

#### Optimization
Now that we have defined the parameter space we want to test, we can continue and perform a cross-validation random search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Initialize the XGBRegressor with default parameters
XGB = XGBRegressor()

# Define the hyperparameters to be tuned
params = {
    'max_depth': [5, 6, 7, 8, 10, 11],
    'learning_rate': [0.005, 0.01, 0.025, 0.05, 0.1, 0.15],
    'colsample_bynode': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # max_features
    'colsample_bylevel': [0.8, 0.9, 1.0],  # max_features per level
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],  # max_features
    'subsample': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],  # max_samples
    'min_child_weight': [5, 10, 15, 20, 25],  # min_samples_leaf
    'reg_alpha': [1, 5, 10, 20, 50],
    'reg_lambda': [0.01, 0.05, 0.1, 0.5, 1.1],    'n_estimators': [1000]
}

# Define RandomizedSearchCV
XGB_cv = RandomizedSearchCV(
    XGB, 
    params, 
    cv=5, 
    n_jobs=-1, 
    verbose=1, 
    n_iter=1000, 
    scoring='neg_mean_absolute_error'
)

# Fit the model
XGB_cv.fit(X_train, Y_train)

# Print the best parameters
print('Tuned XGBoost Parameters: ', XGB_cv.best_params_)

**Note:** In this setup, the eval_set parameter is not directly set in the fit_params since it's not straightforward to use with cross-validation (like RandomizedSearchCV), as each fold will have a different validation set.

**Pro-Tip - Double Search**

If you don't feel confident about setting the various parameter ranges to be tested, do not hesitate to run two optimizations one after the other- the first one with wide parameter ranges; then, a second one performed in more detail around the first optimal values found.

We can now train XGBoostRegressor with RandomizedSearchCV-Optimized Parameters

In [ ]:
# RandomizedSearchCV object with best parameters determined
best_params = XGB_cv.best_params_

# Initialize XGBRegressor with the best parameters
XGB = XGBRegressor(n_jobs=-1, **best_params)

# Assuming fit_params is defined as before, without early stopping rounds and eval_metric
fit_params = {
    'verbose': False
}

# Fit the model with the training data
XGB.fit(x_train, y_train, **fit_params)

# Print the best iteration and score if available
# These attributes are only available if early stopping is used
if hasattr(XGB.get_booster(), 'best_iteration'):
    print(f'Best iteration: {XGB.get_booster().best_iteration}')
if hasattr(XGB.get_booster(), 'best_score'):
    print(f'Best score: {XGB.get_booster().best_score}')

In [ ]:
# Make predictions on the training and test sets
y_train_pred = XGB.predict(X_train)
y_test_pred = XGB.predict(X_test)

In [ ]:
# Define the kpi_ML function
def kpi_ML(Y_train, Y_train_pred, Y_test, Y_test_pred, name=""):
    df = pd.DataFrame(columns=['MAE', 'RMSE', 'Bias'], index=['Train', 'Test'])
    df.index.name = name

    df.loc['Train', 'MAE'] = 100 * np.mean(np.abs(Y_train - Y_train_pred)) / np.mean(Y_train)
    df.loc['Train', 'RMSE'] = 100 * np.sqrt(np.mean((Y_train - Y_train_pred)**2)) / np.mean(Y_train)
    df.loc['Train', 'Bias'] = 100 * np.mean((Y_train - Y_train_pred)) / np.mean(Y_train)

    df.loc['Test', 'MAE'] = 100 * np.mean(np.abs(Y_test - Y_test_pred)) / np.mean(Y_test)
    df.loc['Test', 'RMSE'] = 100 * np.sqrt(np.mean((Y_test - Y_test_pred)**2)) / np.mean(Y_test)
    df.loc['Test', 'Bias'] = 100 * np.mean((Y_test - Y_test_pred)) / np.mean(Y_test)

    df = df.astype(float).round(1)  # Round numbers for display
    print(df)

# Evaluate the model predictions
kpi_ML(Y_train, y_train_pred, Y_test, y_test_pred, name='XGBoost_optimized')

### Multiple Periods Evaluation
Just like AdaBoost, XGBoost unfortunately cannot forecast multiple periods at once, so we will also use scikit-learn’s MultiOutputRegressor. To make the model faster, you should set n_jobs=1 in XGBRegressor and n_jobs=-1 for MultiOutputRegressor. (In other words, a thread will be working independently on each of the future forecast periods, rather than multiple threads working on single forecast periods one by one.)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

# Training and testing
X_train, Y_train, X_test, Y_test = datasets(df, x_len=12, y_len=6, test_loops=12)
XGB = XGBRegressor(n_jobs=1, **best_params)
multi = MultiOutputRegressor(XGB, n_jobs=-1)
multi.fit(X_train, Y_train)

In [ ]:
# Creating the forecast DataFrame with predictions and setting the index
forecast = pd.DataFrame(data=multi.predict(X_test), index=df.index)

print(forecast.head())